Dataset:
  
  https://s3.amazonaws.com/h2o-public-test-data/smalldata/chicago/Chicago_Ohare_International_Airport.csv
  
  https://s3.amazonaws.com/h2o-airlines-unpacked/allyears2k.csv

Use the Chicago Weather and Airlines dataset for the following chanllenge.

   1. Import chicago weather and airlines dataset.
   1. Transform weather data's Date column to three new columns "Year", "Month", "DayofMonth".
   1. Question: What is the range of the Year column in airlines dataset?
   1. Question: What is the range of the Year column in weather dataset?
   1. Filter airlines data to only flights outgoing or incoming to Chicago, ORD.
   1. Run a join of the two frames as you see fit, specify in the comments if you ran an outer left, outer right, or inner join.
   1. Run a GLM model on the resulting merged frame, you can regress on columns such as "IsDepDelayed", "IsArrDelayed", or "WeatherDelay". Pick predictors at your discretion.
   1. Provide some insight on the resulting model which can be a classification or regression model.

In [1]:
#importing libraries
import h2o
h2o.init()
import pandas as pd
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt
sns.set()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: java version "1.8.0_101"; Java(TM) SE Runtime Environment (build 1.8.0_101-b13); Java HotSpot(TM) 64-Bit Server VM (build 25.101-b13, mixed mode)
  Starting server from /Users/mihirzgr8/anaconda/lib/python3.6/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/bc/btz_3q257nb_qgtzwwt7tj1m0000gn/T/tmpgks7giin
  JVM stdout: /var/folders/bc/btz_3q257nb_qgtzwwt7tj1m0000gn/T/tmpgks7giin/h2o_mihirzgr8_started_from_python.out
  JVM stderr: /var/folders/bc/btz_3q257nb_qgtzwwt7tj1m0000gn/T/tmpgks7giin/h2o_mihirzgr8_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321... successful.


H2O cluster uptime:,02 secs
H2O cluster timezone:,America/New_York
H2O data parsing timezone:,UTC
H2O cluster version:,3.22.0.2
H2O cluster version age:,1 month and 19 days
H2O cluster name:,H2O_from_python_mihirzgr8_y4tqht
H2O cluster total nodes:,1
H2O cluster free memory:,1.778 Gb
H2O cluster total cores:,0
H2O cluster allowed cores:,0
H2O cluster status:,"accepting new members, healthy"


In [2]:
#importing the datasets
chicago="https://s3.amazonaws.com/h2o-public-test-data/smalldata/chicago/Chicago_Ohare_International_Airport.csv"
weather=h2o.import_file(path=chicago)
air="https://s3.amazonaws.com/h2o-airlines-unpacked/allyears2k.csv"
airlines=h2o.import_file(path=air)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [3]:
airlines.describe()

Rows:43978
Cols:31




,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,IsArrDelayed,IsDepDelayed
type,int,int,int,int,int,int,int,int,enum,int,enum,int,int,int,int,int,enum,enum,int,int,int,int,enum,int,int,int,int,int,int,enum,enum
mins,1987.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,,1.0,,16.0,17.0,14.0,-63.0,-16.0,,,11.0,0.0,0.0,0.0,,0.0,0.0,0.0,0.0,0.0,0.0,,
mean,1997.5,1.4090909090909094,14.601073263904684,3.820614852880986,1345.8466613820758,1313.2228614307157,1504.6341303788895,1485.2891673109277,,818.8429896766567,,124.81452913540424,125.02156260661894,114.3161110907826,9.31711193698432,10.007390655600114,,,730.1821905650504,5.381368059530624,14.16863418473206,0.024694165264450407,,0.0024785119832643593,4.047800291055633,0.2893764692712415,4.855031904175528,0.017015560282100085,7.620060450016787,,
maxs,2008.0,10.0,31.0,7.0,2400.0,2359.0,2400.0,2359.0,,3949.0,,475.0,437.0,402.0,475.0,473.0,,,3365.0,128.0,254.0,1.0,,1.0,369.0,201.0,323.0,14.0,373.0,,
sigma,6.3443609017107025,1.8747113713439636,9.17579042586145,1.905013119132896,465.3408991242338,476.2511399925997,484.347487903516,492.7504341227009,,777.4043691636348,,73.9744416605902,73.4015946300093,69.63632951506105,29.840221962414834,26.438809042916446,,,578.4380082304242,4.201979939864827,9.905085747204334,0.15519314135784237,,0.049723487218862286,16.205729904484194,4.4167798987341245,18.619776221475686,0.4039401821015118,23.487565874106213,,
zeros,0,0,0,0,0,569,0,569,,0,,0,0,0,1514,6393,,,0,623,557,42892,,43869,7344,8840,7388,8914,7140,,
missing,0,0,0,0,1086,0,1195,0,0,0,32,1195,13,16649,1195,1086,0,0,35,16026,16024,0,9774,0,35045,35045,35045,35045,35045,0,0
0,1987.0,10.0,14.0,3.0,741.0,730.0,912.0,849.0,PS,1451.0,NA,91.0,79.0,nan,23.0,11.0,SAN,SFO,447.0,nan,nan,0.0,NA,0.0,nan,nan,nan,nan,nan,YES,YES
1,1987.0,10.0,15.0,4.0,729.0,730.0,903.0,849.0,PS,1451.0,NA,94.0,79.0,nan,14.0,-1.0,SAN,SFO,447.0,nan,nan,0.0,NA,0.0,nan,nan,nan,nan,nan,YES,NO
2,1987.0,10.0,17.0,6.0,741.0,730.0,918.0,849.0,PS,1451.0,NA,97.0,79.0,nan,29.0,11.0,SAN,SFO,447.0,nan,nan,0.0,NA,0.0,nan,nan,nan,nan,nan,YES,YES


In [4]:
#Split Date column of weather data in 3 columns-year, month, day
weather['Year']=weather['Date'].year()
weather['Month']=weather['Date'].month()
weather['DayofMonth']=weather['Date'].day()

In [5]:
weather.describe()

Rows:1461
Cols:12




,Date,TmaxF,TminF,TmeanF,PrcpIn,SnowIn,CDD,HDD,GDD,Year,Month,DayofMonth
type,time,int,int,real,real,real,real,real,real,int,int,int
mins,1104537600000.0,2.0,-10.0,-2.5,0.0,0.0,0.0,0.0,0.0,2005.0,1.0,1.0
mean,1167609600000.0,59.731006160164306,41.930869267624864,50.83093771389467,0.1258862324814508,0.13482210446631324,2.6830937713894594,16.85215605749488,9.037987679671465,2006.5010266940421,6.5229295003422445,15.729637234770715
maxs,1230681600000.0,102.0,80.0,90.0,6.64,8.8,25.0,67.5,40.0,2008.0,12.0,31.0
sigma,36452046342.55805,21.570103698776933,19.021473406264455,20.00830720652739,0.346369873627173,0.647605252469569,4.817865753839847,16.930054853264178,10.819760641130426,1.1187225040519035,3.449884042297707,8.803105254885079
zeros,0,0,4,0,695,1187,1001,471,704,0,0,0
missing,0,0,0,0,248,140,0,0,0,0,0,0
0,2005-01-01 00:00:00,41.0,25.0,33.0,0.31,0.0,0.0,32.0,0.0,2005.0,1.0,1.0
1,2005-01-02 00:00:00,54.0,33.0,43.5,0.08,0.0,0.0,21.5,0.0,2005.0,1.0,2.0
2,2005-01-03 00:00:00,36.0,32.0,34.0,0.36,0.0,0.0,31.0,0.0,2005.0,1.0,3.0


In [6]:
airlines['Year'].summary()
#Solution: Range(year) in Airlines dataset from 1987-2008

,Year
type,int
mins,1987.0
mean,1997.5
maxs,2008.0
sigma,6.3443609017107025
zeros,0
missing,0
0,1987.0
1,1987.0
2,1987.0


In [7]:
weather['Year'].summary()
#Solution: Range(year) in Weather dataset from 2005-2008

,Year
type,int
mins,2005.0
mean,2006.5010266940421
maxs,2008.0
sigma,1.1187225040519035
zeros,0
missing,0
0,2005.0
1,2005.0
2,2005.0


In [8]:
#Create mask and filter the dataset based on outgoing flights from Chicago ORD
mask = airlines['Origin'] =='ORD'
cols_air = airlines[mask,:]

In [9]:
cols_air.shape

(2246, 31)

In [10]:
#Create mask 2 and filter the dataset based on incoming flights to Chicago ORD
mask2=airlines['Dest']=="ORD"
cols=airlines[mask2,:]

In [11]:
cols.shape

(2103, 31)

In [12]:
#Combine the filtered datasets
filtered=cols_air.rbind(cols)

In [13]:
filtered.shape

(4349, 31)

In [18]:
# Solution: Full outer join on the dataframes
merged=filtered.merge(weather)

In [19]:
merged

Month,Year,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,IsArrDelayed,IsDepDelayed,Date,TmaxF,TminF,TmeanF,PrcpIn,SnowIn,CDD,HDD,GDD
1,2005,1,6,2220,2045,119,2359,UA,544,N325UA,119,134,103,80,95,ORD,BOS,867,2,14,0,,0,58,0,0,0,22,YES,YES,2005-01-01 00:00:00,41,25,33,0.31,0,0,32,0
1,2005,1,6,2342,2115,151,2319,UA,546,N386UA,69,64,42,152,147,ORD,DAY,240,7,20,0,,0,7,0,5,0,140,YES,YES,2005-01-01 00:00:00,41,25,33,0.31,0,0,32,0
1,2005,1,6,1303,1205,1435,1329,UA,547,N849UA,92,84,69,66,58,ORD,MSP,334,8,15,0,,0,0,0,8,0,58,YES,YES,2005-01-01 00:00:00,41,25,33,0.31,0,0,32,0
1,2005,1,6,1213,1215,1344,1353,UA,553,N912UA,91,98,73,-9,-2,ORD,OMA,416,3,15,0,,0,0,0,0,0,0,NO,NO,2005-01-01 00:00:00,41,25,33,0.31,0,0,32,0
1,2005,1,6,646,650,901,916,UA,556,N914UA,75,86,60,-15,-4,ORD,PIT,412,4,11,0,,0,0,0,0,0,0,NO,NO,2005-01-01 00:00:00,41,25,33,0.31,0,0,32,0
1,2005,1,6,1457,1500,1755,1742,UA,559,N531UA,298,282,268,13,-3,ORD,SJC,1829,4,26,0,,0,0,0,0,0,0,YES,NO,2005-01-01 00:00:00,41,25,33,0.31,0,0,32,0
1,2005,1,6,1348,1335,1612,1615,UA,562,N938UA,84,100,62,-3,13,ORD,BUF,473,3,19,0,,0,0,0,0,0,0,NO,YES,2005-01-01 00:00:00,41,25,33,0.31,0,0,32,0
1,2005,1,6,nan,1605,nan,1828,UA,564,000000,nan,83,nan,nan,nan,ORD,CLE,316,0,0,1,A,0,0,0,0,0,0,YES,YES,2005-01-01 00:00:00,41,25,33,0.31,0,0,32,0
1,2005,1,6,2000,1955,2255,2231,UA,565,N342UA,235,216,198,24,5,ORD,SLC,1249,3,34,0,,0,0,0,24,0,0,YES,YES,2005-01-01 00:00:00,41,25,33,0.31,0,0,32,0
1,2005,1,6,1830,1745,2123,2053,UA,566,N932UA,113,128,89,30,45,ORD,ATL,606,5,19,0,,0,15,0,0,0,15,YES,YES,2005-01-01 00:00:00,41,25,33,0.31,0,0,32,0


In [20]:
merged['Year'].describe()

Rows:1087
Cols:1




,Year
type,int
mins,2005.0
mean,2005.0515179392833
maxs,2006.0
sigma,0.22115342082683845
zeros,0
missing,0
0,2005.0
1,2005.0
2,2005.0


In [53]:
# Building a GLM logistic regression to predict "IsDepDelayed" deperature delay
response_columns= "IsDepDelayed"
merged['IsDepDelayed'] = merged['IsDepDelayed'].asfactor()
merged['IsArrDelayed'] = merged['IsArrDelayed'].asfactor()
merged['WeatherDelay'] = merged['WeatherDelay'].asfactor()

predictors = merged.col_names
predictors.remove('IsDepDelayed')


In [54]:
#Split the dataset to train, valid, test
train, valid, test = merged.split_frame(ratios=[.8, .1])

In [55]:
#Intialize and train the model
from h2o.estimators.glm import H2OGeneralizedLinearEstimator
glm_logistic = H2OGeneralizedLinearEstimator(family = "binomial")
glm_logistic.train(x=predictors, y= response_columns, training_frame=train, validation_frame=valid)

glm Model Build progress: |███████████████████████████████████████████████| 100%


In [56]:
glm_logistic.varimp()

In [57]:
#See the model performance
glm_logistic.model_performance()


ModelMetricsBinomialGLM: glm
** Reported on train data. **

MSE: 0.1118133734293014
RMSE: 0.3343850675931887
LogLoss: 0.3720573136600212
Null degrees of freedom: 880
Residual degrees of freedom: 870
Null deviance: 1220.8247171967907
Residual deviance: 655.5649866689573
AIC: 677.5649866689573
AUC: 0.943312019800959
pr_auc: 0.9287108673874223
Gini: 0.886624039601918
Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.407976067637716: 


,NO,YES,Error,Rate
NO,402.0,49.0,0.1086,(49.0/451.0)
YES,63.0,367.0,0.1465,(63.0/430.0)
Total,465.0,416.0,0.1271,(112.0/881.0)


Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.4079761,0.8676123,222.0
max f2,0.2940850,0.8979413,291.0
max f0point5,0.4945753,0.9176863,184.0
max accuracy,0.4838233,0.8751419,187.0
max precision,0.9997481,1.0,0.0
max recall,0.1833172,1.0,370.0
max specificity,0.9997481,1.0,0.0
max absolute_mcc,0.4838233,0.7624908,187.0
max min_per_class_accuracy,0.3869944,0.8627907,230.0
max mean_per_class_accuracy,0.4799566,0.8730135,189.0


Gains/Lift Table: Avg response rate: 48.81 %, avg score: 48.81 %



,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain
,1,0.0102157,0.9995769,2.0488372,2.0488372,1.0,0.9997671,1.0,0.9997671,0.0209302,0.0209302,104.8837209,104.8837209
,2,0.0204313,0.9953187,2.0488372,2.0488372,1.0,0.9978836,1.0,0.9988254,0.0209302,0.0418605,104.8837209,104.8837209
,3,0.0306470,0.9906487,2.0488372,2.0488372,1.0,0.9931273,1.0,0.9969260,0.0209302,0.0627907,104.8837209,104.8837209
,4,0.0408627,0.9880323,2.0488372,2.0488372,1.0,0.9893093,1.0,0.9950218,0.0209302,0.0837209,104.8837209,104.8837209
,5,0.0510783,0.9858844,2.0488372,2.0488372,1.0,0.9868524,1.0,0.9933880,0.0209302,0.1046512,104.8837209,104.8837209
,6,0.1010216,0.9417243,2.0488372,2.0488372,1.0,0.9632975,1.0,0.9785118,0.1023256,0.2069767,104.8837209,104.8837209
,7,0.1509648,0.8850681,2.0488372,2.0488372,1.0,0.9063738,1.0,0.9546466,0.1023256,0.3093023,104.8837209,104.8837209
,8,0.2009081,0.8446340,2.0488372,2.0488372,1.0,0.8684358,1.0,0.9332156,0.1023256,0.4116279,104.8837209,104.8837209
,9,0.3007946,0.6765793,2.0022727,2.0333743,0.9772727,0.7680732,0.9924528,0.8783759,0.2,0.6116279,100.2272727,103.3374287
,10,0.4006810,0.4798825,1.7228858,1.9559721,0.8409091,0.5616980,0.9546742,0.7994307,0.1720930,0.7837209,72.2885835,95.5972067


In [58]:
#Model 2 - Predict Arrival delay "IsArrDelayed" with random forest classifier
response_columns= "IsArrDelayed"
merged['IsDepDelayed'] = merged['IsDepDelayed'].asfactor()
merged['IsArrDelayed'] = merged['IsArrDelayed'].asfactor()
merged['WeatherDelay'] = merged['WeatherDelay'].asfactor()

predictors = merged.col_names

predictors.remove('IsArrDelayed')




In [59]:
# Define model
from h2o.estimators import H2ORandomForestEstimator
model = H2ORandomForestEstimator(ntrees=50, max_depth=20, nfolds=10)

# Train model
model.train(x=predictors, y=response_columns, training_frame=train)

drf Model Build progress: |███████████████████████████████████████████████| 100%


In [60]:
#View the performance of the model on test data
performance = model.model_performance(test_data=test)

print (performance)


ModelMetricsBinomial: drf
** Reported on test data. **

MSE: 0.02255019112958865
RMSE: 0.1501672105673827
LogLoss: 0.11244990769529377
Mean Per-Class Error: 0.008474576271186418
AUC: 0.9995866060355518
pr_auc: 0.7115795269558556
Gini: 0.9991732120711037
Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.45999999999999996: 


,NO,YES,Error,Rate
NO,40.0,1.0,0.0244,(1.0/41.0)
YES,0.0,59.0,0.0,(0.0/59.0)
Total,40.0,60.0,0.01,(1.0/100.0)


Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.4600000,0.9915966,18.0
max f2,0.4600000,0.9966216,18.0
max f0point5,0.61,0.9965636,16.0
max accuracy,0.61,0.99,16.0
max precision,1.0,1.0,0.0
max recall,0.4600000,1.0,18.0
max specificity,1.0,1.0,0.0
max absolute_mcc,0.61,0.9796147,16.0
max min_per_class_accuracy,0.61,0.9830508,16.0
max mean_per_class_accuracy,0.61,0.9915254,16.0


Gains/Lift Table: Avg response rate: 59.00 %, avg score: 59.84 %



,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain
,1,0.17,1.0,1.6949153,1.6949153,1.0,1.0,1.0,1.0,0.2881356,0.2881356,69.4915254,69.4915254
,2,0.31,0.98,1.6949153,1.6949153,1.0,0.9804762,1.0,0.9911828,0.2372881,0.5254237,69.4915254,69.4915254
,3,0.42,0.9,1.6949153,1.6949153,1.0,0.9351515,1.0,0.9765079,0.1864407,0.7118644,69.4915254,69.4915254
,4,0.5,0.8392857,1.6949153,1.6949153,1.0,0.87125,1.0,0.9596667,0.1355932,0.8474576,69.4915254,69.4915254
,5,0.6,0.4209600,1.5254237,1.6666667,0.9,0.7020971,0.9833333,0.9167384,0.1525424,1.0,52.5423729,66.6666667
,6,0.71,0.1800000,0.0,1.4084507,0.0,0.2656727,0.8309859,0.8158691,0.0,1.0,-100.0,40.8450704
,7,0.81,0.1000000,0.0,1.2345679,0.0,0.1322400,0.7283951,0.7314704,0.0,1.0,-100.0,23.4567901
,8,0.91,0.0400000,0.0,1.0989011,0.0,0.0530000,0.6483516,0.6569132,0.0,1.0,-100.0,9.8901099
,9,1.0,0.0,0.0,1.0,0.0,0.0069333,0.59,0.5984150,0.0,1.0,-100.0,0.0


In [61]:
#Variable importance considered by the model
model.varimp()


[('ArrDelay', 3710.740966796875, 1.0, 0.460152125231512),
 ('DepDelay', 745.1849365234375, 0.20081836570949974, 0.09240699776674531),
 ('NASDelay', 613.085693359375, 0.16521921062266784, 0.07602597089709341),
 ('IsDepDelayed', 532.4425659179688, 0.14348685900799352, 0.06602578311532255),
 ('TaxiOut', 351.3150634765625, 0.09467517852096773, 0.043564984603096095),
 ('ActualElapsedTime',
  223.29859924316406,
  0.060176283184734454,
  0.027690244595988858),
 ('LateAircraftDelay',
  185.22206115722656,
  0.04991511474785342,
  0.022968546132399542),
 ('Dest', 170.76779174804688, 0.046019863223019375, 0.021176137864935867),
 ('CarrierDelay',
  153.2832489013672,
  0.04130798950207561,
  0.019007959158421076),
 ('Origin', 128.43612670898438, 0.03461198931917118, 0.015926780443707014),
 ('DepTime', 115.95391082763672, 0.031248182469532104, 0.014378917572977275),
 ('DayofMonth', 115.5536880493164, 0.03114032725088401, 0.014329287764899046),
 ('ArrTime', 113.74711608886719, 0.03065347786511062,

In [62]:
# Model 3- Gradient boosting with one hot encoding to predict Weather Delay "WeatherDelay"
response_columns= "WeatherDelay"
merged['IsDepDelayed'] = merged['IsDepDelayed'].asfactor()
merged['IsArrDelayed'] = merged['IsArrDelayed'].asfactor()
merged['WeatherDelay'] = merged['WeatherDelay'].asfactor()

predictors = merged.col_names

predictors.remove('WeatherDelay')

#split the dataset

train, valid= merged.split_frame(ratios = [.8], seed = 1234)


In [63]:

# initialize the estimator and select the encoding type for categorical columns
from h2o.estimators.gbm import H2OGradientBoostingEstimator
encoding = "one_hot_explicit"

weather_gbm = H2OGradientBoostingEstimator(categorical_encoding = encoding, seed =1234)

# train the model
weather_gbm.train(x = predictors, y = response_columns, training_frame = train, validation_frame = valid)

# print the model performance for the validation frame
weather_gbm.model_performance(valid=True)

gbm Model Build progress: |███████████████████████████████████████████████| 100%

ModelMetricsMultinomial: gbm
** Reported on validation data. **

MSE: 0.018237112651544377
RMSE: 0.1350448542209009
LogLoss: 0.116320171901115
Mean Per-Class Error: 0.3076923076923077
Confusion Matrix: Row labels: Actual class; Column labels: Predicted class



0,5,8,9,10,13,14,17,20,21,22,44,59,Error,Rate
215.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0 / 215
1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1 / 1
0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,nan,0 / 0
0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,nan,0 / 0
0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,nan,0 / 0
1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1 / 1
1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1 / 1
0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,nan,0 / 0
0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,nan,0 / 0
0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,nan,0 / 0


Top-10 Hit Ratios: 


k,hit_ratio
1,0.9817352
2,0.9817352
3,0.9817352
4,0.9817352
5,0.9817352
6,0.9817352
7,0.9817352
8,0.9817352
9,0.9817352
10,0.9817352


In [64]:
#Variable importance considered by the GBM model
weather_gbm.varimp()

[('ArrTime', 3.6979331970214844, 1.0, 0.11992847757158431),
 ('NASDelay', 3.057267427444458, 0.8267503128252687, 0.09915090634896555),
 ('LateAircraftDelay',
  2.9816994667053223,
  0.8063151246504249,
  0.09670014534226769),
 ('PrcpIn', 2.2214157581329346, 0.6007181957538289, 0.07204321866630567),
 ('TaxiOut', 1.8401798009872437, 0.4976238625590706, 0.05967927224000065),
 ('FlightNum', 1.7663944959640503, 0.4776707425074093, 0.05728632492940186),
 ('DepDelay', 1.5183180570602417, 0.4105855828556279, 0.049240903864717044),
 ('CRSDepTime', 1.3685429096221924, 0.3700831888267995, 0.044383513410835226),
 ('CRSArrTime', 1.2546025514602661, 0.33927128604453727, 0.040688288819074846),
 ('SnowIn', 1.2219101190567017, 0.330430555111405, 0.03962803341764429),
 ('AirTime', 1.2069092988967896, 0.32637401342698663, 0.03914153854922632),
 ('DepTime', 1.0898648500442505, 0.2947226983229677, 0.03534564451566284),
 ('Date', 1.0823253393173218, 0.2926838538319419, 0.03510112899984891),
 ('CarrierDelay'

## Insights on the Models 
### Model -1 Logistic Regression for Predicting departure delay
#### This model predicts the departure delay with AUC: 0.943312019800959 

### Model-2 Random forest classifier for Predicting Arrival delay 
#### This model predicts Arrival delay with AUC: 0.9995866060355518 and uses 50 trees to give this prediction

### Model-3 Gradient Boosting to Predict Weather Delay
#### Here each tree is improvised by the previous tree the top hit ratio for this model is 0.9817352

### Further Improvements

Feature engineering, hyperparameter tuning 